In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense

2024-06-04 01:47:12.974227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 01:47:12.974353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 01:47:13.109533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
path = "/kaggle/input/kiemtra/data-kiem-tra-1.csv"
data = pd.read_csv(path)

# Thiết lập cột 'date' làm chỉ số
data.set_index('date', inplace=True)

# Hiển thị dữ liệu
print(data)

            truong_1  truong_2  truong_3  truong_4  truong_5
date                                                        
02.01.2013         0        59     22154    999.00         1
03.01.2013         0        25      2552    899.00         1
05.01.2013         0        25      2552    899.00        -1
06.01.2013         0        25      2554   1709.05         1
15.01.2013         0        25      2555   1099.00         1
...              ...       ...       ...       ...       ...
03.05.2013         4        37      2437    799.00         1
22.05.2013         4        37      2436    799.00         1
03.05.2013         4        37      2436    799.00         1
18.05.2013         4        37      2416    449.00         1
06.05.2013         4        37      2416    449.00         1

[481822 rows x 5 columns]



Chia dữ liệu thành tập huấn luyện và tập kiểm tra

In [3]:
X = data[['truong_1', 'truong_2', 'truong_3', 'truong_4']].values
Y = data['truong_5'].values

In [4]:
split = int(0.8 * len(data))
X_train, X_test = X[:split], X[split:]
Y_train, Y_test = Y[:split], Y[split:]

Reshape dữ liệu cho phù hợp với mô hình

In [5]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

Xây dựng mô hình LSTM

In [6]:
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(units=1))
model_lstm.compile(optimizer='adam', loss='mean_squared_error')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Huấn luyện mô hình LSTM

3012/3012 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step


In [13]:
print("Dự đoán bằng LSTM:", predictions_lstm)

Dự đoán bằng LSTM: [[1.1730269]
 [1.0845747]
 [1.1730269]
 ...
 [1.2041614]
 [1.2041614]
 [1.2041614]]


Sử dụng Kalman filter để dự đoán

In [11]:
from pykalman import KalmanFilter

kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
Y_test_kalman = data['truong_5'].values[split:]  # Lấy các giá trị thực tế trong tập kiểm tra
filtered_state_means, _ = kf.filter(Y_test_kalman)
predictions_kalman = filtered_state_means.flatten()

In [12]:
print("Dự đoán bằng Kalman Filter:", predictions_kalman)

Dự đoán bằng Kalman Filter: [0.5        0.8        0.92307692 ... 1.00001561 1.00000596 1.00000228]
